# RocketPy Simulation for Jose's Rocket

This Jupyter notebook provides a complete setup for simulating the flight of Jose's Rocket using the RocketPy library. The simulation is based on the data extracted from the OpenRocket XML file. The rocket is a single-stage high-power rocket with a fiberglass airframe, carbon fiber fins, and an AeroTech M2500T motor as the default configuration.

The notebook is structured with descriptive markdown sections explaining each major class and component, followed by code cells for implementation. Positions are defined with the coordinate system from tail to nose (0 at the tail bottom). Total rocket length is approximately 2.947 m.

Assumptions:
- Launch site: Spaceport America (latitude 32.99, longitude -106.97, elevation 1400 m).
- Atmospheric model: Standard atmosphere for simplicity (can be changed to forecast).
- Drag coefficient: Constant 0.45 (approximate; can be replaced with a curve from OpenRocket if available).
- Dry mass without motor: Approximately 18 kg (summed from components with approximations for shell masses).
- Center of dry mass: Approximately 1.5 m from tail (estimated).
- Inertia: Approximate values for a cylindrical rocket.
- Motor grain details: Approximated to match propellant mass; actual may vary.

In [ ]:
# Install RocketPy if not installed (uncomment if needed)
# !pip install rocketpy

import rocketpy as rp
from rocketpy import Environment, SolidMotor, Rocket, Flight
import numpy as np
from math import pi

## Environment Configuration

The Environment class in RocketPy defines the launch conditions, including gravity, atmospheric properties, wind, and launch rail details. Here, we set the launch location to Spaceport America, date to October 28, 2025, and use a standard atmospheric model for simplicity. The launch rail length is assumed to be 5 m based on typical high-power rocket setups (adjust as needed). Wind is set to zero for baseline simulation.

In [ ]:
env = rp.Environment(
    latitude=32.99,
    longitude=-106.97,
    elevation=1400,
    date=(2025, 10, 28, 12)  # Hour in UTC
)

# Set atmospheric model (use 'Forecast' with 'GFS' for real data when running)
env.set_atmospheric_model(type='standard')

# Launch rail details
env.set_launch_rail(length=5, azimuth=90, inclination=85)  # 5 m rail, due east, 5 degrees from vertical

# Wind (m/s, direction in degrees)
env.add_wind(0, 0)  # No wind for baseline

env.info()

## Motor Configuration

The SolidMotor class models the rocket motor. We use the AeroTech M2500T reload, with thrust curve from the ENG file. The grain geometry is approximated with 6 grains to match the propellant mass of 4.659 kg and burn time of 4.264 s. The coordinate system is nozzle to combustion chamber. Dry mass is 3.405 kg (casing and empty components). Nozzle and throat radii are approximated.

In [ ]:
# Write the ENG file locally
eng_content = """; AeroTech M2500T
; converted from TMT test stand data 1998 (www.tripoli.org)
; provided by ThrustCurve.org (www.thrustcurve.org)
M2500T 98 751 0 4.6592 8.064 AT
0.082 2651.855
0.249 2780.285
0.416 2820.733
0.583 2843.010
0.751 2847.765
0.918 2851.215
1.084 2854.737
1.252 2861.690
1.420 2858.088
1.586 2851.086
1.754 2844.622
1.922 2830.855
2.089 2804.711
2.255 2765.796
2.423 2710.509
2.591 2648.262
2.757 2586.910
2.925 2520.794
3.093 2462.217
3.259 2419.937
3.426 1894.936
3.594 808.043
3.761 282.403
3.928 97.876
4.096 24.492
4.264 0.000
"""

with open("M2500T.eng", "w") as f:
    f.write(eng_content)

# Define the motor
motor = rp.SolidMotor(
    thrust_source="M2500T.eng",
    burn_time=4.264,
    dry_mass=3.405,
    dry_inertia=(0.5, 0.5, 0.01),  # Approximate (I_xx, I_yy=I_zz, I_xy=0)
    nozzle_radius=0.035,  # Approximate
    throat_radius=0.02,  # Approximate
    grain_number=6,
    grain_density=1750,
    grain_outer_radius=0.044,
    grain_initial_inner_radius=0.019,
    grain_initial_height=0.0905,
    grain_separation=0.001,
    grains_center_of_mass_position=0.3755,
    center_of_dry_mass_position=0.3755,
    nozzle_position=0,
    coordinate_system_orientation="nozzle_to_combustion_chamber"
)

motor.info()

## Rocket Configuration

The Rocket class integrates all components. The airframe is modeled as a cylindrical body with radius 0.0777875 m. We add the nose cone (Haack shape), trapezoidal fins, boat tail transition, parachutes, and positioned masses for internal components like payload, recovery hardware, and bulkheads. Drag is assumed constant. The motor is added at the tail.

Total dry mass without motor is approximated as 18 kg. Center of dry mass is estimated at 1.5 m from the tail. Inertia is approximated for a cylinder-like body.

In [ ]:
# Total length
total_length = 0.381 + 1.0 + 0.0762 + 1.4392 + 0.0508  # 2.9472 m

# Rocket
rocket = rp.Rocket(
    radius=0.0777875,
    mass=18.0,  # Dry mass without motor
    inertia=(80, 80, 0.5),  # Approximate I_xx, I_yy=I_zz, I_xy
    power_off_drag=0.45,
    power_on_drag=0.45,
    center_of_mass_without_motor=1.5,  # From tail
    coordinate_system_orientation="tail_to_nose"
)

# Add motor
rocket.add_motor(motor, position=0.0)  # Nozzle at tail

# Add nose cone (Haack with parameter 0)
nose_base_position = total_length - 0.381
rocket.add_nose(
    length=0.381,
    kind="lvhaack",
    position=total_length,
    shape_parameter=0.0
)

# Add trapezoidal fins
fin_position = 0.0508  # From tail
rocket.add_trapezoidal_fins(
    n=4,
    root_chord=0.3048,
    tip_chord=0.01905,
    span=0.1524,
    sweep_length=0.254,
    cant_angle=0,
    position=fin_position
)

# Add boat tail transition
rocket.add_tail(
    top_radius=0.0777875,
    bottom_radius=0.0635,
    length=0.0508,
    position=0.0508  # Top position from tail
)

# Add parachutes
# Drogue parachute (deploy at apogee)
rocket.add_parachute(
    name="Drogue",
    cd_s= pi * (0.6096 / 2)**2 * 2.2,  # Area * CD
    trigger="apogee",
    sampling_rate=105,
    lag=0,
    noise=(0, 8.3, 0.5)
)

# Main parachute (deploy at 396.24 m altitude)
rocket.add_parachute(
    name="Main",
    cd_s= pi * (2.7432 / 2)**2 * 2.2,
    trigger=396.24,
    sampling_rate=105,
    lag=0,
    noise=(0, 8.3, 0.5)
)

# Add positioned masses (examples; add more as needed with positions from tail)
# Payload at approximate position from tail
payload_position_from_tail = total_length - (0.381 + 0.0381 + 0.4064 / 2)  # Center of packed length
rocket.add_positioned_mass(mass=2.835, position=payload_position_from_tail)

# Telemetry
telemetry_position_from_tail = total_length - (0.381 + 1.0 + 0.2286)  # Approximate in coupler
rocket.add_positioned_mass(mass=0.680, position=telemetry_position_from_tail)

# ... Add other masses similarly with their positions calculated from the XML data

rocket.info()

## Flight Simulation

The Flight class runs the trajectory simulation using the defined environment, rocket, and motor. We simulate the flight with a termination condition at ground hit.

In [ ]:
flight = rp.Flight(
    rocket=rocket,
    environment=env,
    rail_length=5,
    inclination=85,
    heading=90,
    terminate_on_apogee=False
)

flight.all_info()

## Post-Processing and Plots

Generate plots for altitude, velocity, acceleration, and other parameters.

In [ ]:
flight.plots.trajectory_3d()
flight.plots.altitude_time()
flight.plots.velocity_time()